In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hand detection module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Function to recognize gesture
def recognize_gesture(landmarks):
    # Define thumb_tip and other finger tips based on landmarks
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]

    # Check whether each finger is open or closed
    thumb_is_open = thumb_tip[1] < landmarks[3][1]
    index_is_open = index_tip[1] < landmarks[6][1]
    middle_is_open = middle_tip[1] < landmarks[10][1]
    ring_is_open = ring_tip[1] < landmarks[14][1]
    pinky_is_open = pinky_tip[1] < landmarks[18][1]

    # Fist: All fingers are closed
    if not thumb_is_open and not index_is_open and not middle_is_open and not ring_is_open and not pinky_is_open:
        return "Fist"
    
    # Victory: Index and middle fingers are open, others are closed
    if not thumb_is_open and index_is_open and middle_is_open and not ring_is_open and not pinky_is_open:
        return "Victory"
    
    # Thumbs up: Thumb is open, other fingers are closed
    if thumb_is_open and not index_is_open and not middle_is_open and not ring_is_open and not pinky_is_open:
        return "Thumbs Up"

    # Thumbs down (example rule: thumb is down compared to other fingers)
    if thumb_tip[1] > index_tip[1] > middle_tip[1] > ring_tip[1] > pinky_tip[1]:
        return "Thumbs-Down"
        
    # Open hand (e.g., 'Stop'): All fingers are open
    if thumb_is_open and index_is_open and middle_is_open and ring_is_open and pinky_is_open:
        return "Open Hand"

    # Add other gestures here as needed

    # If no specific gesture is recognized
    return "Unknown Gesture"

# Set up MediaPipe Hands
with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # Flip the image horizontally for a later selfie-view display
        image = cv2.flip(image, 1)
        
        # Convert the BGR image to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Process the image and detect hands
        results = hands.process(image_rgb)

        # Draw the hand annotations on the image
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Extract landmarks for gesture recognition
                landmarks = []
                for lm in hand_landmarks.landmark:
                    lmx = int(lm.x * image.shape[1])
                    lmy = int(lm.y * image.shape[0])
                    landmarks.append((lmx, lmy))
                
                # Recognize the gesture
                gesture = recognize_gesture(landmarks)
                
                # Display the gesture
                cv2.putText(image, gesture, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the output
        cv2.imshow('Hand Gesture Detection', image)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


C:\Users\Hamza Rehman\Desktop\Sample\env\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
